In [1]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/ChesapeakeBay/ChesapeakeBayChlorophyll/notebooks/models

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/ChesapeakeBay/ChesapeakeBayChlorophyll/notebooks/models


# Set up

In [70]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt


import logging
from tqdm import tqdm  # For progress bar
# Configure logging instead of print
logging.basicConfig(filename='tuning.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import contextlib
from concurrent.futures import ThreadPoolExecutor, as_completed

In [74]:
# Load tensors
# features_tensor = torch.load('../../data/features_masked_tensor.pt')
# chlorophyll_tensor = torch.load('../../data/chlorophyll_masked_tensor.pt')

features_tensor_dict = {}

for i in range(11):
    # Load tensors using formatted string (f-string)
    features_tensor = torch.load(f'../../data/filesForModel/tensors/features_region{i}_tensor.pt')
    chlorophyll_tensor = torch.load(f'../../data/filesForModel/tensors/chlorophyll_region{i}_tensor.pt')

    # # Attach names to tensor dimensions
    # features_tensor.names = ('time','features','position')
    # chlorophyll_tensor.names = ('time','position')

    # Store the tensors in the dictionary
    features_tensor_dict[f'region_{i}_features'] = features_tensor
    features_tensor_dict[f'region_{i}_chlorophyll'] = chlorophyll_tensor

    print(f"Region {i}: features tensor shape: {features_tensor.shape}, chlorophyll tensor shape: {chlorophyll_tensor.shape}")


Region 0: features tensor shape: torch.Size([2767, 5, 638]), chlorophyll tensor shape: torch.Size([2767, 638])
Region 1: features tensor shape: torch.Size([2767, 5, 450]), chlorophyll tensor shape: torch.Size([2767, 450])
Region 2: features tensor shape: torch.Size([2767, 5, 1472]), chlorophyll tensor shape: torch.Size([2767, 1472])
Region 3: features tensor shape: torch.Size([2767, 5, 1352]), chlorophyll tensor shape: torch.Size([2767, 1352])
Region 4: features tensor shape: torch.Size([2767, 5, 992]), chlorophyll tensor shape: torch.Size([2767, 992])
Region 5: features tensor shape: torch.Size([2767, 5, 975]), chlorophyll tensor shape: torch.Size([2767, 975])
Region 6: features tensor shape: torch.Size([2767, 5, 2856]), chlorophyll tensor shape: torch.Size([2767, 2856])
Region 7: features tensor shape: torch.Size([2767, 5, 486]), chlorophyll tensor shape: torch.Size([2767, 486])
Region 8: features tensor shape: torch.Size([2767, 5, 528]), chlorophyll tensor shape: torch.Size([2767, 5

# Model

We will create create a data stream for each of the 11 regions. Within each of these regions, we use a long short-term memory model (LSTM) to predict the chlorophyll values at each point.

## Defining the classes

In [17]:
import torch
import torch.nn as nn

class RegionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size,h0=None, c0=None):
        super(RegionalLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)


        # Store hidden states if needed
        self.h0 = h0
        self.c0 = c0
    def forward(self, x, h0=None, c0=None, time_batch_size= 100):
        # x shape: (batch_size, time_steps, variables, position)

        batch_size, time_steps, variables, positions = x.size()
        
        # Reshape to (batch_size * position, time_steps, variables) to treat each position separately
        x = x.permute(0, 3, 1, 2).reshape(batch_size * positions, time_steps, variables)
        
        # If no hidden state provided, initialize hidden states
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
            c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)


        # Adjust mini-batching to handle cases where time_steps < time_batch_size
        outputs = []
        for start in range(0, time_steps, time_batch_size):
            end = min(start + time_batch_size, time_steps)
            x_time_batch = x[:, start:end, :]  # Mini-batch along time
            
            # Forward pass through the LSTM with hidden state carryover
            lstm_out, (h0, c0) = self.lstm(x_time_batch, (h0, c0))  # Keep hidden state across time mini-batches
            
            outputs.append(lstm_out)
        
        # Concatenate the outputs for all time mini-batches
        lstm_out = torch.cat(outputs, dim=1)  # Concatenate along the time dimension
        
        # Apply the fully connected layer at every time step for each position
        out = self.fc(lstm_out)  # Shape: (batch_size * positions, time_steps, output_size)
        
        # Reshape back to (batch_size, time_steps, variables, positions)
        out = out.view(batch_size, positions, time_steps).permute(0, 2, 1)
        
        return out, (h0,c0)


Since the relationship between regions comes from a map, we will hard code the data. Simply using a 1 if the regions border each other and 0 if they do not.

In [4]:
# Neighbor mask matrix (as described earlier)
neighbor_mask = torch.tensor([
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Region 1, CB1TF
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],  # Region 2, CB2OH
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # Region 3, CB3MH
    [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],  # Region 4, CB4MH
    [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],  # Region 5, CB5MH
    [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0],  # Region 6, CB6PH
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],  # Region 7, CB7PH
    [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],  # Region 8, CB8PH
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # Region 9, EASMH
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  # Region 10, MOBPH
    [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]   # Region 11, TANMH
], dtype=torch.float32)

# All regions?

In [75]:
train_features_dict = {}
val_features_dict = {}
test_features_dict = {}
train_targets_dict = {}
val_targets_dict = {}
test_targets_dict = {}

# feature shape (time, variables, position)
# chlorophyll shape (time, position)
for region_id in range(11):
    # Access the features and chlorophyll tensors from the dictionary
    features_tensor = features_tensor_dict[f'region_{region_id}_features']
    chlorophyll_tensor = features_tensor_dict[f'region_{region_id}_chlorophyll']

    # reshape to (batch, time, variables, position)
    features_tensor = features_tensor.unsqueeze(0)
    chlorophyll_tensor = chlorophyll_tensor.unsqueeze(0)
    
    # Split data into 70% training, 15% validation, 15% test
    train_size = int(0.7 * features_tensor.shape[1])  # 70% of the time steps
    val_size = int(0.15 * features_tensor.shape[1])   # 15% for validation
    test_size = features_tensor.shape[1] - train_size - val_size  # Remaining for test set

    # Split features into train, validation, and test sets
    train_features = features_tensor[:, :train_size, :, :]  # First 70% for training
    val_features = features_tensor[:, train_size:train_size+val_size, :, :]  # Next 15% for validation
    test_features = features_tensor[:, train_size+val_size:, :, :]  # Remaining for test
    
    # Split chlorophyll targets (same logic)
    train_targets = chlorophyll_tensor[:, :train_size, :]  # First 70% for training
    val_targets = chlorophyll_tensor[:, train_size:train_size+val_size, :]  # Next 15% for validation
    test_targets = chlorophyll_tensor[:, train_size+val_size:, :]  # Remaining for test

    # Store the splits in dictionaries
    # Also correct the indexing
    train_features_dict[f'region_{region_id}'] = train_features
    val_features_dict[f'region_{region_id}'] = val_features
    test_features_dict[f'region_{region_id}'] = test_features
    
    train_targets_dict[f'region_{region_id}'] = train_targets
    val_targets_dict[f'region_{region_id}'] = val_targets
    test_targets_dict[f'region_{region_id}'] = test_targets

    # Print shapes for verification
    print(f"Region {region_id}: Train features {train_features.shape}, Validation features {val_features.shape}, Test features {test_features.shape}")
    print(f"Region {region_id}: Train targets {train_targets.shape}, Validation targets {val_targets.shape}, Test targets {test_targets.shape}")


Region 0: Train features torch.Size([1, 1936, 5, 638]), Validation features torch.Size([1, 415, 5, 638]), Test features torch.Size([1, 416, 5, 638])
Region 0: Train targets torch.Size([1, 1936, 638]), Validation targets torch.Size([1, 415, 638]), Test targets torch.Size([1, 416, 638])
Region 1: Train features torch.Size([1, 1936, 5, 450]), Validation features torch.Size([1, 415, 5, 450]), Test features torch.Size([1, 416, 5, 450])
Region 1: Train targets torch.Size([1, 1936, 450]), Validation targets torch.Size([1, 415, 450]), Test targets torch.Size([1, 416, 450])
Region 2: Train features torch.Size([1, 1936, 5, 1472]), Validation features torch.Size([1, 415, 5, 1472]), Test features torch.Size([1, 416, 5, 1472])
Region 2: Train targets torch.Size([1, 1936, 1472]), Validation targets torch.Size([1, 415, 1472]), Test targets torch.Size([1, 416, 1472])
Region 3: Train features torch.Size([1, 1936, 5, 1352]), Validation features torch.Size([1, 415, 5, 1352]), Test features torch.Size([1,

In [ ]:
model = RegionalLSTM(input_size=6, hidden_size=8, num_layers=1, output_size=1)
criterion = nn.MSELoss()  # Assuming you are using Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training parameters
epochs = 5
batch_size = 1

for i in tqdm(range(11),desc=f"Modelling all regions"):
    # Convert the train_features and train_targets to the appropriate device
    train_features = train_features_dict[f'region_{i}'].to(device)
    train_targets = train_targets_dict[f'region_{i}'].to(device)

    for epoch in tqdm(range(epochs),desc=f"Model for Region {i}"):
        model.train()  # Set model to training mode
        optimizer.zero_grad()

        # Forward pass
        output, _ = model(train_features)

        # Compute the loss
        loss = criterion(output, train_targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

    # Validation step 
    model.eval()  # Set model to evaluation mode
    val_features = val_features_dict[f'region_{i}'].to(device)
    val_targets = val_targets_dict[f'region_{i}'].to(device)

    with torch.no_grad():  # Disable gradient calculation for validation
        val_output, _ = model(val_features)
        val_loss = criterion(val_output, val_targets)
        print(f"Validation Loss: {val_loss.item()}")

# Try sharing

In [76]:
import torch.nn.functional as F

class MultiRegionModel(nn.Module):
    def __init__(self, neighbor_mask, input_size, hidden_size, num_layers, output_size):
        super(MultiRegionModel, self).__init__()
        self.neighbor_mask = neighbor_mask
        self.regions = nn.ModuleList([
            RegionalLSTM(input_size, hidden_size, num_layers, output_size) 
            for _ in range(neighbor_mask.shape[0])
        ])


    def forward(self, region_inputs):
        # region_inputs should be a dictionary where keys are region IDs and values are feature tensors
        region_outputs = {}  # Initialize a dictionary to store outputs for each region

        for region_id, region_data in region_inputs.items():
            # Assuming region_id can be converted to an index
            region_index = int(region_id[-1]) 
            # Initialize combined_h0 for the first pass
            position_size = region_data.shape[-1]
            current_h0 = torch.zeros((batch_size, position_size, self.regions[region_index].lstm.hidden_size))  # Replace with appropriate shape
            
            # Find neighbors
            neighbors = [i for i in range(1, self.neighbor_mask.shape[0] ) if self.neighbor_mask[region_index, i] == 1]
            
            neighbor_hidden_states = []
            for neighbor_index in neighbors:
                _, (neighbor_h0, neighbor_c0) = self.regions[neighbor_index](region_data)  # Use region_data for the neighbor

                neighbor_hidden_states.append(neighbor_h0)
                # Assuming neighbor_hidden_states is a list of (batch_size * positions, time_steps, hidden_size) tensors
            if neighbor_hidden_states:
                aggregated_neighbors_h0 = torch.mean(torch.stack(neighbor_hidden_states), dim=0)  # Mean across neighbors

            # Combine current hidden state with aggregated neighbor hidden states
            combined_h0 = current_h0 + F.interpolate(aggregated_neighbors_h0, size=current_h0.shape[-1], mode='linear', align_corners=False)
            # Adjust this operation as needed

            # Call the RegionalLSTM with the current input and updated hidden states
            output, (current_h0, current_c0) = self.regions[region_index](region_data, combined_h0, neighbor_c0)

            # Store the output for the current region
            region_outputs[region_id] = output# Store the output for the current region
        return region_outputs  # Return the dictionary of outputs for each region


In [77]:
epochs = 1
batch_size=1
input_size=5
hidden_size=2
num_layers=2
#  Initialize the model
model = MultiRegionModel(neighbor_mask,input_size, hidden_size, num_layers, output_size=1)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

last_outputs_dict = {region_id: [] for region_id in train_features_dict.keys()}
validation_losses = {region_id: [] for region_id in val_features_dict.keys()}

for epoch in range(epochs):
    # Create a progress bar for epochs
    
    # Wrap the region_inputs dictionary with tqdm for progress tracking
    with tqdm(total=len(train_features_dict), desc=f"Processing Regions in epoch {epoch}",leave=True) as inner_pbar:
        for region_id, features in train_features_dict.items():

            targets = train_targets_dict[region_id]
            
            # Forward pass: get the outputs for all regions
            region_outputs = model({region_id: features})
            
            # Retrieve output for the specific region
            output = region_outputs[region_id]
            
            # Store for SHAP 
            last_outputs_dict[region_id].append(output.detach().cpu().numpy())  # Detach and move to CPU if necessary

            # Compute loss
            loss = criterion(output, targets)

            # Backpropagation and optimization steps
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            inner_pbar.update(1)  # Increment the inner progress bar by 1

            # Validation step 
            model.eval()  # Set model to evaluation mode
            val_features = val_features_dict[f'region_{i}'].to(device)
            val_targets = val_targets_dict[f'region_{i}'].to(device)

            with torch.no_grad():  # Disable gradient calculation for validation
                val_region_outputs = model({region_id:val_features})
                val_output = val_region_outputs[region_id]
                val_loss = criterion(val_output, val_targets)
                validation_losses[region_id].append(val_loss.item())  # Store the loss in the dictionary



Processing Regions in epoch 0: 100%|██████████| 11/11 [01:23<00:00,  7.58s/it]


In [69]:
# After training, you can concatenate the outputs for SHAP analysis
for region_id, outputs in last_outputs_dict.items():
    last_outputs_dict[region_id] = np.concatenate(outputs, axis=0)  # Combine outputs across epochs if needed


In [ ]:
# Reshape for SHAP
reshaped_region_diict = {}

for region_id, region_data in train_features_dict.item():
    # Example: Aggregate across the position dimension by averaging
    aggregated_data = region_data.mean(dim=3).squeeze(0)  # Shape becomes (time, features)



In [ ]:
import shap

def regional_lstm_predict(region_id, input_data):
    region_model = multi_region_model.regions[region_id]  # Access specific RegionalLSTM
    with torch.no_grad():
        return region_model(input_data).numpy()  # Ensure input_data is in correct shape

# Create the SHAP explainer using the aggregated input data
explainer = shap.KernelExplainer(lambda x: regional_lstm_predict(region_id, x), reshaped_data)

# Calculate SHAP values
shap_values = explainer.shap_values(reshaped_data)

# Visualize the SHAP values
shap.summary_plot(shap_values, reshaped_data)


In [65]:
validation_losses

{'region_0': [0.34159421920776367,
  0.3092031180858612,
  0.2811068892478943,
  0.256698876619339,
  0.23560895025730133,
  0.21779844164848328,
  0.20326048135757446,
  0.1919095814228058,
  0.18349026143550873,
  0.1774870753288269],
 'region_1': [0.18052701652050018,
  0.1787278652191162,
  0.179288849234581,
  0.18036194145679474,
  0.18066148459911346,
  0.17955242097377777,
  0.177238330245018,
  0.17391307651996613,
  0.16899029910564423,
  0.16104598343372345],
 'region_2': [0.5430854558944702,
  0.44662365317344666,
  0.36399152874946594,
  0.29452139139175415,
  0.23653288185596466,
  0.19088152050971985,
  0.16162890195846558,
  0.14997661113739014,
  0.1464603692293167,
  0.14033474028110504],
 'region_3': [1.052377462387085,
  0.9826801419258118,
  0.9204058647155762,
  0.8636765480041504,
  0.8112435340881348,
  0.7619773745536804,
  0.7145645618438721,
  0.6674513220787048,
  0.6189589500427246,
  0.5676072835922241],
 'region_4': [0.1743701845407486,
  0.17460380494594

In [ ]:
output